In [1]:
wd = %pwd
print('Current directory:', wd)
if wd.endswith('models'):
    %cd ..

Current directory: E:\School\ETH\BoobsProject\project\models
E:\School\ETH\BoobsProject\project


In [2]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import tensorflow.contrib.eager as tfe
import sys, os, time, datetime

from models.base import BaseModel
from resources.flags import FLAGS, define_flags

In [3]:
# This can only be run once.
define_flags()

## Read the input data

In this case, MNIST + batch and shuffle it. In our case, it will be quite different.

In [56]:
from tensorflow.examples.tutorials.mnist import input_data

# Reset Tensorflow graph and create new session
tf.reset_default_graph()
session = tf.Session()

def load_data(data_dir):
    """Returns training and test tf.data.Dataset objects."""
    data = input_data.read_data_sets(data_dir, one_hot=True)
    train_ds = tf.data.Dataset.from_tensor_slices((data.train.images,
                                                   data.train.labels))
    test_ds = tf.data.Dataset.from_tensors(
        (data.test.images, data.test.labels))
    return (train_ds, test_ds)

device, data_format = ('/gpu:0', 'channels_first')
if FLAGS.no_gpu or tfe.num_gpus() <= 0:
    device, data_format = ('/cpu:0', 'channels_last')
print('Using device %s, and data format %s.' % (device, data_format))

# Load the datasets
train_ds, test_ds = load_data(FLAGS.in_data_dir)
train_ds = train_ds.shuffle(60000).batch(FLAGS.batch_size)

Using device /cpu:0, and data format channels_last.
Extracting data_in\train-images-idx3-ubyte.gz
Extracting data_in\train-labels-idx1-ubyte.gz
Extracting data_in\t10k-images-idx3-ubyte.gz
Extracting data_in\t10k-labels-idx1-ubyte.gz


In [57]:
# Setup constatnts
IMAGE_PIXELS=784
NOISE_SIZE=100
images_input = tf.placeholder(tf.float32, shape=(None, IMAGE_PIXELS))
noise_input = tf.placeholder(tf.float32, shape=(None, NOISE_SIZE))

In [58]:
## Discriminator
def discriminator(X, var_scope="Discriminator"):
    with tf.variable_scope(var_scope, reuse=tf.AUTO_REUSE):
        # Layer 1
        dx = tf.layers.dense(X, units=1024, use_bias=True, activation=tf.nn.relu)
        # Layer 2
        dx = tf.layers.dense(dx, units=512, use_bias=True, activation=tf.nn.relu)
        # Layer 3
        dx = tf.layers.dense(dx, units=256, use_bias=True, activation=tf.nn.relu)
        # Layer 4
        d_out = tf.layers.dense(dx, units=1, use_bias=True, activation=tf.nn.relu)
        return d_out

In [59]:
## Generator
def generator(X, var_scope='Generator'):
    with tf.variable_scope(var_scope):
        # Layer 1
        gx = tf.layers.dense(X, units=265, use_bias=True, activation=tf.nn.relu)
        # Layer 2
        gx = tf.layers.dense(gx, units=512, use_bias=True, activation=tf.nn.relu)
        # Layer 3
        gx = tf.layers.dense(gx, units=1024, use_bias=True, activation=tf.nn.relu)
        # Layer 4
        g_out = tf.layers.dense(gx, units=784, use_bias=True, activation=tf.nn.relu)
        return g_out

In [60]:
## Losses
g_sample = generator(noise_input)
d_real = discriminator(images_input)
d_fake = discriminator(g_sample)

d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_real, labels=tf.ones_like(d_real)))
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_fake, labels=tf.zeros_like(d_fake)))
d_loss = d_loss_real + d_loss_fake
g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_fake, labels=tf.ones_like(d_fake)))


# Optimizers
d_opt = tf.train.AdamOptimizer(2e-4).minimize(d_loss, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Discriminator'))
g_opt = tf.train.AdamOptimizer(2e-4).minimize(g_loss, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Generator'))

In [61]:
num_test_samples=16
test_noise = tf.random_normal((num_test_samples, NOISE_SIZE))

In [62]:
num_epochs = 200

# Start interactive session
session = tf.InteractiveSession()
# Init Variables
tf.global_variables_initializer().run()

In [63]:
def next_batch()

# Iterate through epochs
for epoch in range(FLAGS.epochs):
    for n_batch, (batch,_) in enumerate(train_ds.make_initializable_iterator()):
        # 1. Train Discriminator
        print(batch.shape)
        #X_batch = images_to_vectors(batch.permute(0, 2, 3, 1).numpy())
        feed_dict = {images_input: batch, noise_input: tf.random_normal(BATCH_SIZE, NOISE_SIZE)}
        _, d_error, d_pred_real, d_pred_fake = session.run(
            [d_opt, d_loss, d_real, d_fake], feed_dict=feed_dict
        )

        # 2. Train Generator
        feed_dict = {noise_input: noise(BATCH_SIZE, NOISE_SIZE)}
        _, g_error = session.run(
            [g_opt, g_loss], feed_dict=feed_dict
        )

        if n_batch % 1 == 0:
        #    display.clear_output(True)
            # Generate images from test noise
            test_images = session.run(
                G_sample, feed_dict={Z: test_noise}
            )
        #    test_images = vectors_to_images(test_images)
        #    # Log Images
        #    logger.log_images(test_images, num_test_samples, epoch, n_batch, num_batches, format='NHWC');
        #    # Log Status
        #    logger.display_status(
        #        epoch, num_epochs, n_batch, num_batches,
        #        d_error, g_error, d_pred_real, d_pred_fake
        #    )

TypeError: 'Iterator' object is not iterable

(<tf.Tensor 'IteratorGetNext_1:0' shape=(?, 784) dtype=float32>, <tf.Tensor 'IteratorGetNext_1:1' shape=(?, 10) dtype=float64>)
